<div class="alert alert-info">
<b>"Сквозная аналитика для Салона красоты"</b> 
</div>

In [1]:
# Импортируем необходимые библиотеки для анализа
import pandas as pd 
import numpy as np
from datetime import datetime
from datetime import timedelta

In [2]:
# Загрузим датасеты для анализа - данные по рекламе
df_ads = pd.read_csv('C:\\Users\\HP\\Desktop\\DataAnalitics\\ads.csv', sep=',', na_values='?', encoding='cp1251')
df_ads.head()

,created_at,d_ad_account_id,d_utm_source,d_utm_medium,d_utm_campaign,d_utm_content,d_utm_term,m_clicks,m_cost
0,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8404700756,NaN,2.0,48.840
1,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476161,NaN,3.0,100.776
2,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476162,NaN,1.0,10.140
3,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476163,NaN,0.0,0.000
4,2022-08-08,xo-for-client-ya,yandex,cpc,48306435,8813476164,NaN,0.0,0.000


In [3]:
# Посмотрим основную информацию по датасету.
df_ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8602 entries, 0 to 8601
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   created_at       8602 non-null   object 
 1   d_ad_account_id  8602 non-null   object 
 2   d_utm_source     8602 non-null   object 
 3   d_utm_medium     8602 non-null   object 
 4   d_utm_campaign   8602 non-null   int64  
 5   d_utm_content    8602 non-null   int64  
 6   d_utm_term       0 non-null      float64
 7   m_clicks         8602 non-null   float64
 8   m_cost           8602 non-null   float64
dtypes: float64(3), int64(2), object(4)
memory usage: 605.0+ KB


In [4]:
# Приведем метки к формату обьект.
df_ads['d_utm_campaign'] = df_ads.d_utm_campaign.astype('object')
df_ads['d_utm_content'] = df_ads.d_utm_content.astype('object')

In [5]:
# Загрузим данные по лидам, сразу меняем пропуски в метках на 'not set' и сортируем по дате
df_leads = pd.read_csv('C:\\Users\\HP\\Desktop\\DataAnalitics\\leads.csv', sep=',', encoding='cp1251').fillna('not set').sort_values(by='lead_created_at', ascending=True)

# Оставим только те данные, где рекламный истоник - yandex
df_leads = df_leads[df_leads['d_lead_utm_source'] == 'yandex']
df_leads.head()

,lead_created_at,lead_id,d_lead_utm_source,d_lead_utm_medium,d_lead_utm_campaign,d_lead_utm_content,d_lead_utm_term,client_id
8965,2022-01-03,40cee5db-6ccb-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701661,not set,not set
9364,2022-01-04,3255c8b6-6d2d-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701664,not set,8aa2b25a-bdd7-11e9-abb5-c412f533dba1
14928,2022-01-05,368f2030-6de0-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701661,not set,7eb80bd0-78bf-11e9-aba2-c412f533dba1
14854,2022-01-06,93c30660-6ed0-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701664,not set,9eb261e7-49ef-11ec-9052-848f69e142cf
9703,2022-01-06,bedc5d15-6e97-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701661,not set,7581512e-6ec5-11ec-9056-848f69e142cf


In [7]:
# Загрузим данные по оплатам, отсортируем по дате оплаты
df_pay = pd.read_csv('C:\\Users\\HP\\Desktop\\DataAnalitics\\purchases.csv', sep=',', na_values='?', encoding='cp1251').sort_values(by='purchase_created_at', ascending=True)
# удалим строки с нулевыми продажами
df_pay = df_pay.loc[df_pay['m_purchase_amount'] != 0.0]
df_pay.head()

,purchase_created_at,purchase_id,client_id,m_purchase_amount
50193,2022-01-01,ddad6dec-1b19-474e-8512-605357c01ae6,25f9d4ce-d5a3-11e9-abbe-c412f533dba1,1200.0
31716,2022-01-01,2af0d074-3e53-471b-8a10-23a8a5c6b043,d6294a24-cd98-11e9-abba-c412f533dba1,1200.0
3224,2022-01-02,115d5dbb-317c-4d47-afce-1e8fb9151453,57e70aec-dfbf-11eb-ac44-c412f533dba1,40.0
8973,2022-01-02,94486ddd-34a4-4823-9975-cf6f54798484,d7a211f4-6876-11ec-9056-848f69e142cf,500.0
25776,2022-01-02,3f354288-07db-4a99-8028-4969b7fb1513,8bbd52c3-2cb5-11ec-904d-848f69e142cf,120.0


In [8]:
# Подтянем метки к таблице с оплатами
df_pay_utm = df_pay.merge(df_leads, how='left', on=['client_id'])

# Изменим тип данных на дату
df_pay_utm['purchase_created_at'] = pd.to_datetime(df_pay_utm['purchase_created_at'])
df_pay_utm['lead_created_at'] = pd.to_datetime(df_pay_utm['lead_created_at'])

# Добавим новый столбец period, посчитаем период между датой создания заявки и оплаты
df_pay_utm['period'] = df_pay_utm['purchase_created_at'] - df_pay_utm['lead_created_at']

# Оставим только те оплаты, где период не более 15 дней
df_pay_utm = df_pay_utm[df_pay_utm['period'] < '16 days']
df_pay_utm = df_pay_utm[df_pay_utm['period'] > '0 days']

df_pay_utm.head()

,purchase_created_at,purchase_id,client_id,m_purchase_amount,lead_created_at,lead_id,d_lead_utm_source,d_lead_utm_medium,d_lead_utm_campaign,d_lead_utm_content,d_lead_utm_term,period
733,2022-01-08,41b8d8e1-4489-4afc-a20d-565990f589c9,7eb80bd0-78bf-11e9-aba2-c412f533dba1,290.0,2022-01-05,368f2030-6de0-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701661,not set,3 days
821,2022-01-09,84c83f95-1d8b-4cab-9397-20624964d110,8aa2b25a-bdd7-11e9-abb5-c412f533dba1,290.0,2022-01-04,3255c8b6-6d2d-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701664,not set,5 days
980,2022-01-10,a323aa49-1d15-4638-b3e9-8c1c3119c109,3cbb1a2f-7129-11ec-9057-848f69e142cf,50.0,2022-01-09,43de5963-7104-11ec-9057-848f69e142cf,yandex,cpc,48306450,8404701661,not set,1 days
1128,2022-01-10,d55ab1ca-88a2-4766-858b-5519b1fd7df4,fd5c8272-d0a2-11e9-abbd-c412f533dba1,290.0,2022-01-07,a4480dc9-6fa0-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701665,not set,3 days
1208,2022-01-10,f68a364e-4a31-44f8-9206-677b4f42d649,7581512e-6ec5-11ec-9056-848f69e142cf,290.0,2022-01-06,bedc5d15-6e97-11ec-9056-848f69e142cf,yandex,cpc,48306450,8404701661,not set,4 days


In [9]:
# добавим в эту таблицу новый столбец со сцепкой меток
df_pay_utm['concat'] = df_pay_utm[['lead_created_at', 'd_lead_utm_source', 'd_lead_utm_medium', 'd_lead_utm_campaign']].astype(str).agg('/'.join, axis=1)

In [10]:
# Сделаем сводную таблицу по оплатам, чтобы объединить покупки по меткам
df_pivot_pay = df_pay_utm.pivot_table(index='concat', 
                                  aggfunc={'purchase_id': 'count', 'm_purchase_amount' : 'sum'}).reset_index()
# переименуем колонки
df_pivot_pay.rename(columns = {'purchase_id' : 'quantity', 'm_purchase_amount' : 'summa'}, inplace = True)
df_pivot_pay.tail()

,concat,summa,quantity
232,2022-10-05/yandex/cpc/48306450,7999.0,1
233,2022-10-06/yandex/cpc/48306435,10990.0,1
234,2022-10-10/yandex/cpc/78599498,7999.0,1
235,2022-10-11/yandex/cpc/48306435,6799.0,1
236,2022-10-12/yandex/cpc/48306435,11990.0,1


In [11]:
# Видим, что под правило 15 дней попадают оплаты только на 2.3 млн руб, 
# остальные оплаты либо были сделаны раньше или позже заявок, либо по другим меткам.
df_pivot_pay.summa.sum()

2254988.0

In [12]:
# Сделаем сводную таблицу по расходам
df_pivot_cost = df_ads.pivot_table(index=['created_at', 'd_utm_source', 'd_utm_medium', 'd_utm_campaign'], values=['m_clicks', 'm_cost'], aggfunc='sum').round(2).reset_index()

# добавим новый столбец со сцепкой даты и меток
df_pivot_cost['concat'] = df_pivot_cost[['created_at', 'd_utm_source', 'd_utm_medium', 'd_utm_campaign']].astype(str).agg('/'.join, axis=1)
df_pivot_cost.tail()

,created_at,d_utm_source,d_utm_medium,d_utm_campaign,m_clicks,m_cost,concat
1277,2022-09-14,yandex,cpc,48306450,48.0,1856.21,2022-09-14/yandex/cpc/48306450
1278,2022-09-14,yandex,cpc,48306473,1.0,33.44,2022-09-14/yandex/cpc/48306473
1279,2022-09-14,yandex,cpc,48306487,10.0,228.06,2022-09-14/yandex/cpc/48306487
1280,2022-09-14,yandex,cpc,48306494,0.0,0.00,2022-09-14/yandex/cpc/48306494
1281,2022-09-14,yandex,cpc,48306518,0.0,0.00,2022-09-14/yandex/cpc/48306518


In [13]:
# Сделаем сводную таблицу по лидам
df_pivot_lead = df_leads.pivot_table(index=['lead_created_at', 'd_lead_utm_source', 'd_lead_utm_medium', 'd_lead_utm_campaign'], values='lead_id', aggfunc='count').reset_index()

# добавим новый столбец со сцепкой даты и меток
df_pivot_lead['concat'] = df_pivot_lead[['lead_created_at', 'd_lead_utm_source', 'd_lead_utm_medium', 'd_lead_utm_campaign']].astype(str).agg('/'.join, axis=1)
df_pivot_lead.head()

,lead_created_at,d_lead_utm_source,d_lead_utm_medium,d_lead_utm_campaign,lead_id,concat
0,2022-01-03,yandex,cpc,48306450,1,2022-01-03/yandex/cpc/48306450
1,2022-01-04,yandex,cpc,48306450,1,2022-01-04/yandex/cpc/48306450
2,2022-01-05,yandex,cpc,48306450,1,2022-01-05/yandex/cpc/48306450
3,2022-01-06,yandex,cpc,48306450,3,2022-01-06/yandex/cpc/48306450
4,2022-01-07,yandex,cpc,48306450,6,2022-01-07/yandex/cpc/48306450


In [14]:
# Объединим таблицы по расходам и лидам
df_merge = df_pivot_cost.merge(df_pivot_lead[['lead_id', 'concat']], how='left', on=['concat']).fillna(0).sort_values(by='created_at', ascending=True)
df_merge.head()

,created_at,d_utm_source,d_utm_medium,d_utm_campaign,m_clicks,m_cost,concat,lead_id
0,2022-01-03,yandex,cpc,48306435,1.0,11.95,2022-01-03/yandex/cpc/48306435,0.0
1,2022-01-03,yandex,cpc,48306450,10.0,190.13,2022-01-03/yandex/cpc/48306450,1.0
2,2022-01-04,yandex,cpc,48306435,1.0,11.36,2022-01-04/yandex/cpc/48306435,0.0
3,2022-01-04,yandex,cpc,48306450,29.0,617.62,2022-01-04/yandex/cpc/48306450,1.0
4,2022-01-05,yandex,cpc,48306435,0.0,0.00,2022-01-05/yandex/cpc/48306435,0.0


In [15]:
# Проверяем, что все данные подтянулись
# Видим, что рекламные кампании yandex - привели на сайт 1582 лида (всего за этот период было 25 тыс лидов)
print(df_merge.m_cost.sum())
print(df_merge.m_clicks.sum())
print(df_merge.lead_id.sum())

451141.10000000003
18797.0
1582.0


In [16]:
# Подтянем оплаты к объединенной таблице 
df_merge = df_merge.merge(df_pivot_pay[['quantity', 'summa', 'concat']], how='left', on=['concat']).fillna(0)
df_merge.head()

,created_at,d_utm_source,d_utm_medium,d_utm_campaign,m_clicks,m_cost,concat,lead_id,quantity,summa
0,2022-01-03,yandex,cpc,48306435,1.0,11.95,2022-01-03/yandex/cpc/48306435,0.0,0.0,0.0
1,2022-01-03,yandex,cpc,48306450,10.0,190.13,2022-01-03/yandex/cpc/48306450,1.0,0.0,0.0
2,2022-01-04,yandex,cpc,48306435,1.0,11.36,2022-01-04/yandex/cpc/48306435,0.0,0.0,0.0
3,2022-01-04,yandex,cpc,48306450,29.0,617.62,2022-01-04/yandex/cpc/48306450,1.0,2.0,9990.0
4,2022-01-05,yandex,cpc,48306435,0.0,0.00,2022-01-05/yandex/cpc/48306435,0.0,0.0,0.0


In [17]:
# Данный рекламный источник принес компании около 2 млн руб.
df_merge.summa.sum()

1995399.0

In [18]:
# Добавим два столбца с метриками
df_merge['CPL'] = (df_merge['m_cost'] / df_merge['lead_id']).round(2)
df_merge['ROAS'] = (df_merge['summa'] / df_merge['m_cost']).round(2)

In [19]:
# Приведем таблицу в читабельный вид и выгрузим отчет
df_merge.drop('concat', axis=1, inplace=True)
df_merge.rename(columns = {'created_at' : 'Дата',
                           'd_utm_source' : 'UTM source', 
                           'd_utm_medium' : 'UTM medium', 
                           'd_utm_campaign' : 'UTM campaign',
                           'm_clicks' : 'Количество кликов', 
                           'm_cost' : 'Расходы на рекламу', 
                           'lead_id' : 'Количество лидов', 
                           'quantity' : 'Количество покупок', 
                           'summa' : 'Выручка от продаж'}, inplace = True)
df_merge

,Дата,UTM source,UTM medium,UTM campaign,Количество кликов,Расходы на рекламу,Количество лидов,Количество покупок,Выручка от продаж,CPL,ROAS
0,2022-01-03,yandex,cpc,48306435,1.0,11.95,0.0,0.0,0.0,inf,0.00
1,2022-01-03,yandex,cpc,48306450,10.0,190.13,1.0,0.0,0.0,190.13,0.00
2,2022-01-04,yandex,cpc,48306435,1.0,11.36,0.0,0.0,0.0,inf,0.00
3,2022-01-04,yandex,cpc,48306450,29.0,617.62,1.0,2.0,9990.0,617.62,16.17
4,2022-01-05,yandex,cpc,48306435,0.0,0.00,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1277,2022-09-14,yandex,cpc,48306473,1.0,33.44,0.0,0.0,0.0,inf,0.00
1278,2022-09-14,yandex,cpc,48306494,0.0,0.00,0.0,0.0,0.0,NaN,NaN
1279,2022-09-14,yandex,cpc,48306435,14.0,449.03,2.0,1.0,9999.0,224.52,22.27
1280,2022-09-14,yandex,cpc,48306450,48.0,1856.21,4.0,1.0,10400.0,464.05,5.60


In [20]:
df_merge.fillna(0).to_csv('analysis_test.csv')